In [46]:
import ccxt
import pandas as pd
import os

# GLOBAL VARIABLES
# Initialize the specific exchange 
EXCHANGE_NAME = "kraken"
EXCHANGE = getattr(ccxt, EXCHANGE_NAME)()  # i.e. ccxt.kraken()
markets = EXCHANGE.load_markets() 

TICKER_DATA_PATH = r"C:\Users\Damja\CODING_LOCAL\trading\ticker_specific_data\data_updated"

def fetch_ohlcv_data_per_symbol(symbol, timeframe='1h', **kwargs):
    try:
        # Fetch OHLCV data for the current pairs
        ohlcv = EXCHANGE.fetch_ohlcv(symbol, timeframe=timeframe, **kwargs)
        # Convert to a DataFrame
        df = pd.DataFrame(ohlcv, columns=['Date', 'open', 'high', 'low', 'close', 'volume'])
        df['Date'] = pd.to_datetime(df['Date'], unit='ms')
        df.set_index('Date', inplace=True)
        df['usd_volume'] = df['close'] * df['volume']
        return df
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")


def update_ohlcv_data_per_symbol(symbol, timeframe='1h', timestamp=None, **kwargs):
    '''
    Update data from last 
    arguments:
        symbol: str (as given by exchange, i.e. 'BTC/USD', needs to be converted to 'BTCUSD' for data folder)
        timeframe: str
        timestamp: datetime
        kwargs: other arguments for fetch_ohlcv_data_per_symbol
    '''
    assert timestamp is not None, "Timestamp must be provided"
    data_folder_symbol = symbol.replace('/', '')
    if timeframe == '1h':
        data_folder_symbol = data_folder_symbol + '_60'
    else:
        raise Exception("Timeframe not supported")

    df = load_ohlcv_data_per_symbol(data_folder_symbol)
    df_since_timestamp = fetch_ohlcv_data_per_symbol(symbol=symbol, timeframe='1h', since=int(timestamp.timestamp()*1000))
    df_new = pd.concat([df, df_since_timestamp]).drop_duplicates(keep='first').reset_index(drop=True)    
    df.index = pd.to_datetime(df.index)
    return df_new
        

def save_ohlcv_data_per_symbol(df, symbol):
    df.to_csv(TICKER_DATA_PATH + f"/{symbol}.csv", index=False)


def load_ohlcv_data_per_symbol(symbol):
    pd.read_csv(TICKER_DATA_PATH + f"/{symbol}.csv")


In [3]:
# update data in the TICKER_DATA_PATH_OUTPUT folder
# check if same timeframe is applied
NUM_PAIRS_TO_LOAD = 100
TICKER_DATA_PATH_OUTPUT = r"C:\Users\Damja\CODING_LOCAL\trading\ticker_specific_data\data_updated"

pairs = pd.read_csv("pairs.csv")
pairs = pairs.iloc[:NUM_PAIRS_TO_LOAD, 0].values
TIMEFRAME = '60'


# load each pair
for symbol in pairs:
    data_folder_symbol = symbol.replace("/", "")
    data_folder_symbol = data_folder_symbol + "_" + TIMEFRAME

    # check if file in ticker_specific_data exists
    if not os.path.exists(f'{TICKER_DATA_PATH_OUTPUT}\\{data_folder_symbol}.csv'):
        print(f"file for {symbol} does not exist")
        continue
    else:
        print(f"Changing file for symbol {symbol}")
        try:
            df = pd.read_csv(f'{TICKER_DATA_PATH_OUTPUT}\\{data_folder_symbol}.csv', header=None, index_col=0)
        except pd.errors.EmptyDataError:
            print(f"file for {symbol} is empty")
            continue


file for BTC/USD does not exist
Changing file for symbol PEPE/USD
Changing file for symbol XRP/USD
Changing file for symbol WIF/USD
Changing file for symbol USDT/USD
file for DOGE/USD does not exist
Changing file for symbol SOL/USD
Changing file for symbol EUR/USD
Changing file for symbol TURBO/USD
Changing file for symbol ETH/USD
Changing file for symbol SUI/USD
Changing file for symbol XLM/USD
Changing file for symbol WOO/USD
Changing file for symbol NEAR/USD
Changing file for symbol FTM/USD
Changing file for symbol SHIB/USD
Changing file for symbol USDC/USD
Changing file for symbol LINK/USD
Changing file for symbol BONK/USD
Changing file for symbol POPCAT/USD
Changing file for symbol GALA/USD
Changing file for symbol W/USD
Changing file for symbol TAO/USD
Changing file for symbol ALGO/USD
Changing file for symbol FET/USD
Changing file for symbol GBP/USD
Changing file for symbol TRX/USD
Changing file for symbol SUSHI/USD
file for KAS/USD does not exist
Changing file for symbol FLOKI/

In [41]:
for symbol in pairs[0:3]:
    data_folder_symbol = symbol.replace("/", "")
    data_folder_symbol = data_folder_symbol + "_" + TIMEFRAME

    # check if file in ticker_specific_data exists
    if not os.path.exists(f'{TICKER_DATA_PATH_OUTPUT}\\{data_folder_symbol}.csv'):
        print(f"file for {symbol} does not exist")
        continue
    else:
        print(f"Changing file for symbol {symbol}")
        try:
            df = pd.read_csv(f'{TICKER_DATA_PATH_OUTPUT}\\{data_folder_symbol}.csv', index_col=0)
        except pd.errors.EmptyDataError:
            print(f"file for {symbol} is empty")
            continue

file for BTC/USD does not exist
Changing file for symbol PEPE/USD
Changing file for symbol XRP/USD


In [42]:
df.index

Index(['2019-09-29 23:00:00', '2019-09-30 00:00:00', '2019-09-30 01:00:00',
       '2019-09-30 02:00:00', '2019-09-30 03:00:00', '2019-09-30 04:00:00',
       '2019-09-30 05:00:00', '2019-09-30 06:00:00', '2019-09-30 07:00:00',
       '2019-09-30 08:00:00',
       ...
       '2024-09-30 14:00:00', '2024-09-30 15:00:00', '2024-09-30 16:00:00',
       '2024-09-30 17:00:00', '2024-09-30 18:00:00', '2024-09-30 19:00:00',
       '2024-09-30 20:00:00', '2024-09-30 21:00:00', '2024-09-30 22:00:00',
       '2024-09-30 23:00:00'],
      dtype='object', name='Date', length=43800)

In [44]:
df.index= pd.to_datetime(df.index)

In [47]:
update_ohlcv_data_per_symbol(symbol="XRP/USD", timeframe="1h", timestamp=df.index[-1])

AttributeError: 'NoneType' object has no attribute 'index'

In [38]:
df.index[-1]

Timestamp('2024-09-30 23:00:00')